In [2]:
import os
import pandas as pd
import spacy 
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.tokenize import RegexpTokenizer
stop_words = stopwords.words('english')
import re
import sklearn
from nltk.stem.wordnet import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
from nltk.corpus import stopwords
from nltk.tag import pos_tag
from nltk import FreqDist, classify, NaiveBayesClassifier

import re, string, random

In [3]:
df=pd.read_csv("data/Womens_Clothing_E-Commerce_Reviews.csv")

### Cleaning and Pre-Processing the Dataset

#### Removing Null values

In [4]:
df = df[['Review Text','Recommended IND']].dropna()

In [5]:
positive_review = []
negative_review = []
for index, row in df.iterrows():
    if row['Recommended IND'] == 1:
        positive_review.append(row['Review Text'])
    else:
        negative_review.append(row['Review Text'])

In [6]:
def remove_noise(tweet_tokens, stop_words = ()):

    cleaned_tokens = []

    for token, tag in pos_tag(tweet_tokens):
        token = re.sub('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+#]|[!*\(\),]|'\
                       '(?:%[0-9a-fA-F][0-9a-fA-F]))+','', token)
        token = re.sub("(@[A-Za-z0-9_]+)","", token)

        if tag.startswith("NN"):
            pos = 'n'
        elif tag.startswith('VB'):
            pos = 'v'
        else:
            pos = 'a'

        lemmatizer = WordNetLemmatizer()
        token = lemmatizer.lemmatize(token, pos)

        if len(token) > 0 and token not in string.punctuation and token.lower() not in stop_words:
            cleaned_tokens.append(token.lower())
    return cleaned_tokens

In [7]:
def get_all_words(cleaned_tokens_list):
    for tokens in cleaned_tokens_list:
        for token in tokens:
            yield token

In [8]:
def get_tweets_for_model(cleaned_tokens_list):
    for tweet_tokens in cleaned_tokens_list:
        yield dict([token, True] for token in tweet_tokens)

In [9]:
stop_words=stopwords.words('english')

positive_tweet_tokens = []
negative_tweet_tokens = []

for sent in positive_review:
    positive_tweet_token=word_tokenize(sent)
    positive_tweet_tokens.append(positive_tweet_token)
    
for sent in negative_review:
    negative_tweet_token=word_tokenize(sent)
    negative_tweet_tokens.append(negative_tweet_token)

positive_cleaned_list=[]
negative_cleaned_list=[]

for tokens in positive_tweet_tokens:
    positive_cleaned_list.append(remove_noise(tokens, stop_words))
    
for tokens in negative_tweet_tokens:
    negative_cleaned_list.append(remove_noise(tokens, stop_words))
    
all_pos_words = get_all_words(positive_cleaned_list)

freq_dist_pos = FreqDist(all_pos_words)

positive_tokens_for_model = get_tweets_for_model(positive_cleaned_list)
negative_tokens_for_model = get_tweets_for_model(negative_cleaned_list)

positive_dataset = [(tweet_dict, "Positive") for tweet_dict in positive_tokens_for_model]

negative_dataset = [(tweet_dict, "Negative") for tweet_dict in negative_tokens_for_model]

dataset = positive_dataset + negative_dataset

random.shuffle(dataset)

train_data = dataset[:20000]
test_data = dataset[20000:]

classifier = NaiveBayesClassifier.train(train_data)

print("Accuracy is:", classify.accuracy(classifier, test_data))

Accuracy is: 0.87731919727376


In [10]:
custom_tweet = "I had such high hopes for this dress and really wanted it to work for me. i initially ordered the petite small (my usual size) but i found this to be outrageously small. so small in fact that i could not zip it up! i reordered it in petite medium, which was just ok. overall, the top half was comfortable and fit nicely, but the bottom half had a very tight under layer and several somewhat cheap (net) over layers. imo, a major design flaw was the net over layer sewn directly into the zipper - it c"

custom_tokens = remove_noise(word_tokenize(custom_tweet))

print(custom_tweet, classifier.classify(dict([token, True] for token in custom_tokens)))

I had such high hopes for this dress and really wanted it to work for me. i initially ordered the petite small (my usual size) but i found this to be outrageously small. so small in fact that i could not zip it up! i reordered it in petite medium, which was just ok. overall, the top half was comfortable and fit nicely, but the bottom half had a very tight under layer and several somewhat cheap (net) over layers. imo, a major design flaw was the net over layer sewn directly into the zipper - it c Negative
